In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Pytorch'

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# torchvision.datasets을 통해 데이터 로딩
training_data = datasets.FashionMNIST(
                        root='data',
                        train=True,
                        download=True,
                        transform=ToTensor()
)

test_data = datasets.FashionMNIST(
                        root='data',
                        train=False,
                        download=True,
                        transform=ToTensor()
)

In [ ]:
# 모듈명 그대로 클래스 타입
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [ ]:
# transform을 적용하지 않고 데이터 로딩
a = datasets.FashionMNIST(
                        root='data',
                        train=True,
                        download=True
)

In [ ]:
# for문으로 train_data의 data 하나만 끄집어내기
for i in training_data:
  print(type(i))
  print(type(i[0]))
  # print(i)
  # normalized tensor 출력
  break

<class 'tuple'>
<class 'torch.Tensor'>


In [ ]:
# transform 하지 않은 data 하나만 불러오기
for i in a:
  print(type(i))
  print(i)
  break

<class 'tuple'>
(<PIL.Image.Image image mode=L size=28x28 at 0x7FE8E005A750>, 9)


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(type(X))
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

<class 'torch.Tensor'>
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# device를 가져온다
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
                              nn.Linear(28*28, 512),
                              nn.ReLU(),
                              nn.Linear(512, 512),
                              nn.ReLU(),
                              nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

# model = NeuralNetwork()
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss : {loss:>7f} [{current/size*100:>0.2f}%]')

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5

for t in range(epochs):
  print(f'epoch : {t+1} \n-------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(train_dataloader, model, loss_fn)

print('Done')

epoch : 1 
-------------------
loss : 2.296475 [0.00%]
loss : 2.290001 [10.67%]
loss : 2.272003 [21.33%]
loss : 2.271149 [32.00%]
loss : 2.254327 [42.67%]
loss : 2.217104 [53.33%]
loss : 2.239146 [64.00%]
loss : 2.195008 [74.67%]
loss : 2.191325 [85.33%]
loss : 2.169972 [96.00%]
Test Error: 
 Accuracy: 36.8%, Avg loss: 2.160582 

epoch : 2 
-------------------
loss : 2.165943 [0.00%]
loss : 2.161316 [10.67%]
loss : 2.109559 [21.33%]
loss : 2.126264 [32.00%]
loss : 2.074802 [42.67%]
loss : 2.007754 [53.33%]
loss : 2.044101 [64.00%]
loss : 1.961218 [74.67%]
loss : 1.968656 [85.33%]
loss : 1.900320 [96.00%]
Test Error: 
 Accuracy: 52.6%, Avg loss: 1.897308 

epoch : 3 
-------------------
loss : 1.925087 [0.00%]
loss : 1.902054 [10.67%]
loss : 1.797043 [21.33%]
loss : 1.833885 [32.00%]
loss : 1.719819 [42.67%]
loss : 1.667509 [53.33%]
loss : 1.692731 [64.00%]
loss : 1.593347 [74.67%]
loss : 1.621183 [85.33%]
loss : 1.515342 [96.00%]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.532386 

epoc

In [ ]:
torch.save(model.state_dict(), 'model.pth')
print("Saved Pytorch Model State to model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

# 실습

In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
training_ds = datasets.FashionMNIST(
                root='/data',
                train=True,
                transform=ToTensor(),
                download=True
)

test_ds = datasets.FashionMNIST(
                root='/data',
                train=False,
                transform=ToTensor(),
                download=True
)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_ds, batch_size=batch_size)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [ ]:
from torch.nn.modules.activation import Softmax
class NeuralNetwork(nn.Module):
  sw = True
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.network = nn.Sequential(
                      nn.Linear(28*28, 512),
                      nn.ReLU(),
                      nn.Linear(512, 512),
                      nn.ReLU(),
                      nn.Linear(512, 10),
                      nn.Softmax(1)
    )
  
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.network(x)
    if NeuralNetwork.sw:
      print(logits)
      NeuralNetwork.sw=False
    return logits     

In [ ]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (network): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss = loss.item()
      current = len(X) * batch
      print(f'loss : {loss} [{current/size*100:>0.2f}%]')

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  print(f'Test Error\n Accuracy: {correct/size*100}% Avg Loss: {test_loss/num_batches}')

In [ ]:
epochs = 1

for t in range(epochs):
  print(f'Epochs : {t+1}\n ---------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
  print()

Epochs : 1
 ---------------------
tensor([[5.6447e-08, 1.2161e-10, 1.8968e-06, 4.5210e-06, 3.6435e-06, 1.0162e-04,
         1.5413e-07, 3.7817e-03, 1.7264e-02, 9.7884e-01],
        [9.8889e-01, 5.1186e-06, 2.9729e-03, 7.3989e-03, 5.2729e-04, 2.8796e-06,
         8.1555e-06, 5.4051e-08, 1.9063e-04, 4.2817e-06],
        [7.8851e-02, 4.5782e-01, 1.6309e-02, 3.8612e-01, 4.3845e-02, 3.5496e-03,
         6.4014e-03, 1.9078e-03, 2.8980e-03, 2.3029e-03],
        [5.3822e-01, 8.0083e-02, 6.4182e-02, 2.3714e-01, 7.4543e-02, 6.7570e-04,
         2.6074e-03, 7.5743e-05, 2.2305e-03, 2.4937e-04],
        [2.6143e-02, 8.1044e-02, 5.4564e-04, 8.8720e-01, 4.5827e-03, 7.4173e-05,
         1.1711e-04, 2.5372e-05, 1.9114e-04, 7.3015e-05],
        [1.0228e-02, 4.2577e-06, 8.1238e-01, 6.6282e-04, 1.6585e-01, 2.6885e-05,
         5.0910e-05, 2.2024e-06, 1.0778e-02, 1.4518e-05],
        [8.0685e-06, 1.5653e-05, 9.5397e-05, 4.3480e-04, 3.7253e-04, 3.4518e-03,
         9.2965e-05, 9.7797e-01, 1.1971e-02, 5.5867

In [ ]:
NeuralNetwork.sw = True
NeuralNetwork.sw

True

# ㄹㅇ 실습

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
train_ds = datasets.FashionMNIST(
                      path,
                      train=True,
                      transform=ToTensor(),
                      download=True
)
test_ds = datasets.FashionMNIST(
                      path,
                      train=False,
                      transform=ToTensor(),
                      download=True
)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_ds, batch_size=batch_size)
test_dataloader = DataLoader(test_ds, batch_size=batch_size)

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
                              nn.Linear(28*28, 512),
                              nn.ReLU(),
                              nn.Linear(512, 512),
                              nn.ReLU(),
                              nn.Linear(512, 10)                              
    )

  
  def forward(self, x):
    x = self.flatten(x)
    return self.linear_relu_stack(x)

In [ ]:
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)

  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      current = batch*len(X)
      print(f'Loss: {loss:>0.3f}  [{current}/{size}]')

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)

      pred = model(X)
      loss = loss_fn(pred, y)

      test_loss += loss.item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  print(f'Test Error ')
  print(f'Accuracy: {correct/size*100:>0.3f}%')
  print(f'Avg Loss: {test_loss/num_batches:>0.3f}')
  print()

In [ ]:
epochs = 5

for t in range(epochs):  
  print(f'Epochs {t+1} -----------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

Epochs 1 -----------------
Loss: 2.300  [0/60000]
Loss: 2.300  [6400/60000]
Loss: 2.273  [12800/60000]
Loss: 2.268  [19200/60000]
Loss: 2.274  [25600/60000]
Loss: 2.216  [32000/60000]
Loss: 2.240  [38400/60000]
Loss: 2.202  [44800/60000]
Loss: 2.200  [51200/60000]
Loss: 2.176  [57600/60000]
Test Error 
Accuracy: 0.372
Avg Loss: 2.168

Epochs 2 -----------------
Loss: 2.174  [0/60000]
Loss: 2.177  [6400/60000]
Loss: 2.117  [12800/60000]
Loss: 2.132  [19200/60000]
Loss: 2.110  [25600/60000]
Loss: 2.023  [32000/60000]
Loss: 2.063  [38400/60000]
Loss: 1.988  [44800/60000]
Loss: 1.994  [51200/60000]
Loss: 1.931  [57600/60000]
Test Error 
Accuracy: 0.557
Avg Loss: 1.926

Epochs 3 -----------------
Loss: 1.950  [0/60000]
Loss: 1.933  [6400/60000]
Loss: 1.820  [12800/60000]
Loss: 1.857  [19200/60000]
Loss: 1.776  [25600/60000]
Loss: 1.700  [32000/60000]
Loss: 1.730  [38400/60000]
Loss: 1.635  [44800/60000]
Loss: 1.658  [51200/60000]
Loss: 1.554  [57600/60000]
Test Error 
Accuracy: 0.594
Avg Lo

In [ ]:
shape = (2,3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.5604, 0.3820, 0.1909],
        [0.6197, 0.4382, 0.9409]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")


Random Tensor: 
 tensor([[0.7296, 0.7968, 0.2383],
        [0.9799, 0.1103, 0.3133]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


torch.Size([2, 3])

In [ ]:
print(rand_tensor.shape)

torch.Size([2, 3])


In [ ]:
device

'cuda'

In [ ]:
t = torch.rand(62, 3, 28, 28, dtype=torch.float, device=device)
s = nn.Flatten()(t)
s.size()

torch.Size([62, 2352])

In [ ]:
t = torch.rand(3, 28, 28, dtype=torch.float, device=device)
s = nn.Flatten()(t)
s.size()

torch.Size([3, 784])